<a href="https://colab.research.google.com/github/Mahjabeenqamar1/Machain-Learning/blob/main/rats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 21.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3095441 sha256=4662f9203959a4415165037bc84cd98d6937340f780dbf96fa5b0d9dad9f001c
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from surprise import Dataset, Reader
from surprise.model_selection import train_test_split

In [ ]:
import pandas as pd

In [ ]:
ratings = pd.read_csv('rats.csv') # reading data in pandas df

In [ ]:
ratings_dict = {'itemID': list(ratings.itemId),
                'userID': list(ratings.userId),
                'rating': list(ratings.rating)}
df = pd.DataFrame(ratings_dict)


In [ ]:
# A reader is still needed but only the rating_scale param is required.
# The Reader class is used to parse a file containing ratings.
reader = Reader(rating_scale=(0.5, 5.0))


In [ ]:
# Load the dataset
df = pd.read_csv('/content/rats.csv')

In [ ]:
df.head(10)

,itemID,userID,rating,User_Item_Count
0,0,13,1.533462,5
1,0,15,1.283205,6
2,0,17,1.506836,5
3,0,19,1.260289,5
4,0,23,3.203168,1
5,0,24,2.645116,2
6,0,29,1.326969,3
7,0,44,4.293818,3
8,0,47,1.640807,3
9,0,59,2.807194,4


In [ ]:
# The columns must correspond to user id, item id and ratings (in that order).
data = Dataset.load_from_df(df[['userId', 'itemId', 'rating']], reader)


In [ ]:
# Split data into 5 folds

# data.split(n_folds=5)

from surprise import NMF
from surprise.model_selection import cross_validate
# from surprise import NMF


In [ ]:
# Split the dataset into training and testing sets
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)


In [ ]:
# NMF
model = NMF()
model.fit(trainset)

In [ ]:

cross_validate(model, data, measures=['RMSE'], cv=5, verbose=True)

Evaluating RMSE of algorithm NMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9427  0.9455  0.9637  0.9574  0.9451  0.9509  0.0082  
Fit time          15.51   14.98   15.32   15.45   19.96   16.24   1.87    
Test time         0.71    1.64    0.48    0.41    0.69    0.79    0.44    


{'test_rmse': array([0.94270387, 0.94551977, 0.96368589, 0.95736075, 0.94505843]),
 'fit_time': (15.51027798652649,
  14.976129531860352,
  15.32239055633545,
  15.450026273727417,
  19.96026873588562),
 'test_time': (0.7117011547088623,
  1.6406152248382568,
  0.48247790336608887,
  0.4107391834259033,
  0.6943795680999756)}

In [ ]:
# Make predictions on the test set
predictions = model.test(testset)


In [ ]:
# Extract the predicted ratings and the ground truth ratings
y_pred = [round(pred.est) for pred in predictions]
y_true = [int(pred.r_ui) for pred in predictions]

In [ ]:
# Evaluate the performance of the model using classification metrics
# Accuracy
accuracy = accuracy_score(y_true, y_pred)

In [ ]:
# Precision
precision = precision_score(y_true, y_pred, average='macro')

In [ ]:

# Recall
recall = recall_score(y_true, y_pred, average='macro')

# F1 score
f1 = f1_score(y_true, y_pred, average='macro')

# Print the classification metrics
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 score: {f1}")


Accuracy: 0.509366766076961
Precision: 0.4568482357921189
Recall: 0.43169890003402484
F1 score: 0.4170254360365995


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
